In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# Model

In [2]:
class Autoencoder(nn.Module):
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder1 = nn.Conv2d(3, 16, 3, padding=1)
        self.encoder2 = nn.Conv2d(16, 8, 3, padding=1)
        self.encoder3 = nn.Conv2d(8, 4, 3, padding=1)
        self.encoder4 = nn.Conv2d(4, 4, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, ceil_mode=True)
        self.up1 = nn.Upsample(75, mode='nearest')
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        
        self.decoder1 = nn.Conv2d(4, 4, 3, padding=1)
        self.decoder2 = nn.Conv2d(4, 4, 3, padding=1)
        self.decoder3 = nn.Conv2d(4, 8, 3, padding=1)
        self.decoder4 = nn.Conv2d(8, 16, 3, padding=1)
        self.decoder5 = nn.Conv2d(16, 3, 3, padding=1)
        
    def forward(self, x):
        x = F.relu(self.encoder1(x))
        x = self.pool(x)
        x = F.relu(self.encoder2(x))
        x = self.pool(x)
        x = F.relu(self.encoder3(x))
        x = self.pool(x)
        x = F.relu(self.encoder4(x))
        encoder = self.pool(x)
        
        x = F.relu(self.decoder1(encoder))
        x = self.up1(x)
        x = F.relu(self.decoder2(x))
        x = self.up(x)
        x = F.relu(self.decoder3(x))
        x = self.up(x)
        x = F.relu(self.decoder4(x))
        x = self.up(x)
        decoder = F.relu(self.decoder5(x))
        
        return decoder

In [3]:
model = Autoencoder()
model

Autoencoder(
  (encoder1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (up1): Upsample(size=75, mode=nearest)
  (up): Upsample(scale_factor=2.0, mode=nearest)
  (decoder1): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder2): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder5): Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [4]:
with torch.no_grad():
    model.eval()
    img = torch.rand(6,3,600,600)
    print(model(img).shape)

torch.Size([6, 3, 600, 600])


# Dataset

In [5]:
import os
import glob

from torch.utils.data import DataLoader, Dataset
from PIL import Image

class myDataset(Dataset):
    
    def __init__(self, image_path, transform=None):
        self.image_path = image_path
        self.transform = transform
        self.image_list = glob.glob(image_path + '/*')
        
    def __len__(self):
        
        return len(self.image_list)
    
    def __getitem__(self, idx):
        file_name = self.image_list[idx]
        
        image = Image.open(file_name)
        
        if self.transform:
            image = self.transform(image)
            
        return image, image  

def collate_fn(batch):
    return tuple(zip(*batch))

In [6]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((600,600)),
    transforms.Grayscale(3),
    transforms.ToTensor()
])

train_path = r'C:\Users\gjust\Documents\Github\data\fruit\apple'
test_path = r'C:\Users\gjust\Documents\Github\data\fruit\apple'

trainset = myDataset(train_path, transform=transform)
testset = myDataset(test_path, transform=transform)

trainloader = DataLoader(trainset, batch_size=4)
testloader = DataLoader(testset, batch_size=4)

inputs, outputs = iter(trainloader).next()

# GPU

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(device)

cuda


# Optimizer

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.MSELoss()
loss_func

MSELoss()

# 학습

In [12]:
EPOCH = 5
train_epoch_loss = []
val_peoch_loss = []

for e in range(EPOCH):
    
    # Train
    model.train()
    train_iter_loss = []
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_iter_loss.append(loss.item())
        
    train_epoch_loss.append(sum(train_iter_loss))
    print(f'Train Epoch[{e+1}/{EPOCH}] / Loss : {loss}')
    
    # Validation
    
    if e+1 == 5:
        model.eval()
        val_iter_loss = []
        for i, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_func(outputs, labels)
            val_iter_loss.append(loss.item())

        val_peoch_loss.append(sum(val_iter_loss))
        print(f'Validation Epoch[{e+1}/{EPOCH}] / Loss : {loss}')

Train Epoch[1/5] / Loss : 0.005480456631630659
Train Epoch[2/5] / Loss : 0.00514173461124301
Train Epoch[3/5] / Loss : 0.005242711864411831
Train Epoch[4/5] / Loss : 0.005308091174811125
Train Epoch[5/5] / Loss : 0.004775404930114746
Validation Epoch[5/5] / Loss : 0.004396789241582155
